In [38]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import marvin

# Necessary Marvin code
from marvin import config
config.setDR('DR17')
config.switchSasUrl(sasmode='mirror')

In [39]:
# Open data
drpall = fits.open('/Users/Jonah/sas/dr17/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits')
tbdata = drpall[1].data

# Prints column names
tbdata.columns.names

[WARNING]: unclosed file <_io.BufferedReader name='/Users/Jonah/sas/dr17/manga/spectro/redux/v3_1_1/drpall-v3_1_1.fits'> (ResourceWarning)


['plate',
 'ifudsgn',
 'plateifu',
 'mangaid',
 'versdrp2',
 'versdrp3',
 'verscore',
 'versutil',
 'versprim',
 'platetyp',
 'srvymode',
 'objra',
 'objdec',
 'ifuglon',
 'ifuglat',
 'ifura',
 'ifudec',
 'ebvgal',
 'nexp',
 'exptime',
 'drp3qual',
 'bluesn2',
 'redsn2',
 'harname',
 'frlplug',
 'cartid',
 'designid',
 'cenra',
 'cendec',
 'airmsmin',
 'airmsmed',
 'airmsmax',
 'seemin',
 'seemed',
 'seemax',
 'transmin',
 'transmed',
 'transmax',
 'mjdmin',
 'mjdmed',
 'mjdmax',
 'gfwhm',
 'rfwhm',
 'ifwhm',
 'zfwhm',
 'mngtarg1',
 'mngtarg2',
 'mngtarg3',
 'catidnum',
 'plttarg',
 'manga_tileid',
 'nsa_iauname',
 'ifudesignsize',
 'ifutargetsize',
 'ifudesignwrongsize',
 'z',
 'zmin',
 'zmax',
 'szmin',
 'szmax',
 'ezmin',
 'ezmax',
 'probs',
 'pweight',
 'psweight',
 'psrweight',
 'sweight',
 'srweight',
 'eweight',
 'esweight',
 'esrweight',
 'nsa_field',
 'nsa_run',
 'nsa_camcol',
 'nsa_version',
 'nsa_nsaid',
 'nsa_nsaid_v1b',
 'nsa_z',
 'nsa_zdist',
 'nsa_sersic_absmag',
 'nsa_e

In [40]:
# Finding the index, plateifu and drp3qual of galaxy
plateifu_list =['11950-6102', '8600-12703', '8466-6103', '8726-1901', '8721-12701', '8481-6101', 
            '8725-1902', '8078-12703', '8615-6104', '8597-1902', '8485-1901']

for i in range(len(plateifu_list)):

    ind = np.where(tbdata['plateifu'] == plateifu_list[i])[0][0]
    print("index: ", ind)

    plateifu = tbdata["plateifu"][ind]
    print("plateifu", plateifu)

    drp3qual = tbdata["drp3qual"][ind]
    print("drp3qual: ", drp3qual, "\n")

index:  2456
plateifu 11950-6102
drp3qual:  1073746242 

index:  7520
plateifu 8600-12703
drp3qual:  0 

index:  6834
plateifu 8466-6103
drp3qual:  64 

index:  8287
plateifu 8726-1901
drp3qual:  0 

index:  8197
plateifu 8721-12701
drp3qual:  0 

index:  6883
plateifu 8481-6101
drp3qual:  0 

index:  8271
plateifu 8725-1902
drp3qual:  0 

index:  4775
plateifu 8078-12703
drp3qual:  4096 

index:  7736
plateifu 8615-6104
drp3qual:  2 

index:  7490
plateifu 8597-1902
drp3qual:  0 

index:  6945
plateifu 8485-1901
drp3qual:  0 



Test code to select 'perfect' sources for selection (i.e. ```drp3qual``` = 0)

In [36]:
# Initially empty list to contain filtered ids
filtered_ids = []

# Initial count variable
count = 0

# Ensure count is less than length of plateifu_list
while count < len(plateifu_list):
    
    # print("count: ", count)
    
    # Select plateifu from sample
    plateifu = plateifu_list[count]
    print("plateifu: ", plateifu)
    
    # Get index of galaxy
    ind = np.where(tbdata['plateifu'] == plateifu)[0][0]
    print("index: ", ind)
    
    # Find drp3qual
    drp3qual = tbdata["drp3qual"][ind]
    print("drp3qual: ", drp3qual)
    
    # Limit indexes to those with drp3qual equal to 0 (and no duplicates)
    if drp3qual == 0 and id not in filtered_ids:
        
        print("Filtered\n")
        filtered_ids.append(ind)
        
    else:
        print("Not filtered\n")
    
    count += 1
    
print("Final list: ", filtered_ids)
print("Fraction accepted: ", str(np.round(len(filtered_ids)/len(plateifu_list), decimals=2)))

plateifu:  11950-6102
index:  2456
drp3qual:  1073746242
Not filtered

plateifu:  8600-12703
index:  7520
drp3qual:  0
Filtered

plateifu:  8466-6103
index:  6834
drp3qual:  64
Not filtered

plateifu:  8726-1901
index:  8287
drp3qual:  0
Filtered

plateifu:  8721-12701
index:  8197
drp3qual:  0
Filtered

plateifu:  8481-6101
index:  6883
drp3qual:  0
Filtered

plateifu:  8725-1902
index:  8271
drp3qual:  0
Filtered

plateifu:  8078-12703
index:  4775
drp3qual:  4096
Not filtered

plateifu:  8615-6104
index:  7736
drp3qual:  2
Not filtered

plateifu:  8597-1902
index:  7490
drp3qual:  0
Filtered

plateifu:  8485-1901
index:  6945
drp3qual:  0
Filtered

Final list:  [7520, 8287, 8197, 6883, 8271, 7490, 6945]
Fraction accepted:  0.64


Write code to randomly select 100 galaxies that satisfy data quality criteria

In [52]:
# Array to store selected plate-ifudesign values
plateifu_list = []

# Total number of manga galaxies
total = len(tbdata['plateifu'])
print(total)

# Ensure only 100 galaxies are sampled
while len(plateifu_list) < 100:
    
    # Pick random galaxy index
    ind = np.random.randint(low=0, high=total, dtype=int)
    
    # Find DRP3QUAL value for given index
    drp3qual = tbdata["drp3qual"][ind]
    
    # Filter for DRP3QUAL equal to 0 and not in plate_ifudesign_list already
    if drp3qual == 0 and ind not in plateifu_list:
        
        # Append index for use in 100 galaxy sample
        plateifu_list.append(ind)
    
print("length: ", len(plateifu_list))
print(plateifu_list)

11273
length:  100
[11126, 988, 6037, 5102, 10972, 10175, 8100, 2002, 9382, 7392, 6696, 5952, 1066, 5905, 6693, 694, 7020, 10189, 8304, 627, 5552, 7600, 8901, 10200, 1378, 2917, 5217, 9250, 5170, 9768, 7667, 9375, 750, 10129, 1964, 6107, 5911, 7126, 5088, 5338, 6051, 10564, 6472, 7404, 1145, 314, 8196, 4438, 913, 5642, 8376, 9915, 6414, 4492, 9333, 2901, 5930, 4858, 5219, 6528, 9709, 2790, 6769, 9811, 9978, 10724, 7683, 7236, 5065, 10763, 10056, 11246, 5849, 1411, 5751, 8017, 2234, 287, 8216, 6038, 5196, 10263, 10268, 8055, 7477, 4814, 7482, 10294, 7087, 5483, 11027, 6768, 10875, 10152, 6916, 6212, 6801, 9912, 8905, 9055]


Save plateifu_list as .txt file

In [54]:
np.savetxt("100_galaxy_plateifu_list.txt", plate_ifudesign_list, fmt="%d")